In [1]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


In [2]:
import pandas as pd
import numpy as np
import pickle
import tqdm
import random
import time

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import *
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
with open('../Data/raw_mod_4_negative_sample.pickle', 'rb') as f:
    data = pickle.load(f)

In [5]:
sparse_features = ['sa_id', 'album_id', 'buy_seg']
dense_features = ['release_date', 'run_time', 'movie_meta_price', 'i30_meta_price', 'buy_history_price','buy_tot',
                  'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4']
# ambiguous_features = [ 'agree_yn', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small']
# unnecessary_features = ['category_id', # NaN 약 650만 중에 25만 정도.
#                         's_time', 'e_time', # future
#                         'watch_duration', # future
#                          # \\N 값이 약 482만 개
#                         'vod_s_point', 'vod_e_point', 
#                         'album_name',
#                         'view_no',
#                         'fod', 'buy_1_2', 'buy_3',
#                         'amt_r_gabun',
#                         'watch_ratio',
#                         'weekdays', 'weekends', 'dawn', 'morning', 'afternoon', 'evening',
#                         'current_rate']
target = ['label']

In [6]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                           for i,feat in enumerate(sparse_features)]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [7]:
train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

In [8]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )

In [ ]:
history = model.fit(train_model_input, train[target].values,
                    batch_size=2**10, epochs=200, verbose=2, validation_split=0.2, )

Epoch 1/200


/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


5780/5780 - 79s - loss: 0.3130 - binary_crossentropy: 0.3021 - val_loss: 0.2995 - val_binary_crossentropy: 0.2831
Epoch 2/200
5780/5780 - 78s - loss: 0.2867 - binary_crossentropy: 0.2655 - val_loss: 0.2922 - val_binary_crossentropy: 0.2700
Epoch 3/200
5780/5780 - 78s - loss: 0.2734 - binary_crossentropy: 0.2476 - val_loss: 0.2949 - val_binary_crossentropy: 0.2680
Epoch 4/200
5780/5780 - 122s - loss: 0.2633 - binary_crossentropy: 0.2354 - val_loss: 0.2955 - val_binary_crossentropy: 0.2659
Epoch 5/200
5780/5780 - 112s - loss: 0.2597 - binary_crossentropy: 0.2306 - val_loss: 0.2964 - val_binary_crossentropy: 0.2658
Epoch 6/200
5780/5780 - 83s - loss: 0.2566 - binary_crossentropy: 0.2269 - val_loss: 0.2975 - val_binary_crossentropy: 0.2662
Epoch 7/200
5780/5780 - 83s - loss: 0.2544 - binary_crossentropy: 0.2243 - val_loss: 0.2958 - val_binary_crossentropy: 0.2640
Epoch 8/200
5780/5780 - 83s - loss: 0.2527 - binary_crossentropy: 0.2222 - val_loss: 0.2958 - val_binary_crossentropy: 0.2638
Ep

Epoch 66/200
5780/5780 - 113s - loss: 0.2450 - binary_crossentropy: 0.2129 - val_loss: 0.2977 - val_binary_crossentropy: 0.2636
Epoch 67/200
5780/5780 - 113s - loss: 0.2450 - binary_crossentropy: 0.2129 - val_loss: 0.2965 - val_binary_crossentropy: 0.2624
Epoch 68/200
5780/5780 - 113s - loss: 0.2451 - binary_crossentropy: 0.2129 - val_loss: 0.2964 - val_binary_crossentropy: 0.2622
Epoch 69/200
5780/5780 - 100s - loss: 0.2451 - binary_crossentropy: 0.2129 - val_loss: 0.2963 - val_binary_crossentropy: 0.2623
Epoch 70/200
5780/5780 - 113s - loss: 0.2449 - binary_crossentropy: 0.2128 - val_loss: 0.2985 - val_binary_crossentropy: 0.2644
Epoch 71/200
5780/5780 - 113s - loss: 0.2450 - binary_crossentropy: 0.2128 - val_loss: 0.2966 - val_binary_crossentropy: 0.2624
Epoch 72/200
5780/5780 - 113s - loss: 0.2450 - binary_crossentropy: 0.2128 - val_loss: 0.2967 - val_binary_crossentropy: 0.2626
Epoch 73/200
5780/5780 - 100s - loss: 0.2450 - binary_crossentropy: 0.2128 - val_loss: 0.2971 - val_bina

In [42]:
pred_ans = model.predict(test_model_input, batch_size=32)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


test LogLoss nan
test AUC 0.9264
